**Import required libraries and scripts**

In [4]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


In [5]:
software = Path('/home/tony/DockM8/software')
protein_file = Path('/home/tony/DockM8/wocondock_main/protein.pdb')
ref_file = Path('/home/tony/DockM8/wocondock_main/ref.sdf')
docking_library = Path('/home/tony/DockM8/wocondock_main/library.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS', 'LinF9', 'SCORCH', 'Vinardo', 'PLECnn', 'NNScore', 'KORPL', 'ConvexPLR', 'RTMScore', 'AAScore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
ncpus = 5
pocket = 'reference'

#Create a temporary folder for all further calculations
w_dir = protein_file.parent / protein_file.stem
print('The working directory has been set to:', w_dir)
(w_dir).mkdir(exist_ok=True)

if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, protein_file)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')
    print(pocket_definition)

The working directory has been set to: /home/tony/DockM8/wocondock_main/protein

[2023-Oct-24 12:07:25]: Extracting pocket from /home/tony/DockM8/wocondock_main/protein.pdb using /home/tony/DockM8/wocondock_main/ref.sdf as reference ligand

[2023-Oct-24 12:07:43]: Finished extracting pocket from /home/tony/DockM8/wocondock_main/protein.pdb using /home/tony/DockM8/wocondock_main/ref.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:631: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if pd_version < LooseVersion("0.17.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:641: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  elif pd_version < LooseVersion("0.23.0"):


In [6]:
if (w_dir / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, w_dir, id_column, software, protonation, ncpus)


[2023-Oct-24 12:09:08]: Standardizing docking library using ChemBL Structure Pipeline...


Standardizing molecules: 100%|██████████| 10/10 [00:00<00:00, 267.31mol/s]


[2023-Oct-24 12:09:08]: Standardization of compound library finished: Started with 10, ended with 10 : 0 compounds lost

OSError: File error: Bad output file /home/tony/DockM8/wocondock_main/temp/standardized_library.sdf

In [ ]:
docking(w_dir, protein_file, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5)
concat_all_poses(w_dir, docking_programs)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir / 'clustering/'f'{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)


In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'LinF9'])